In [1]:
import time

import numpy as np
import tensorflow as tf

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('albaniantext') as f:
    text = f.read()


In [3]:
from collections import Counter

In [4]:
words = text.split()
    
# Remove all words with  5 or fewer occurences
word_counts = Counter(words)
trimmed_words = [word for word in words if word_counts[word] > 5]

In [5]:
print("Total words: {}".format(len(trimmed_words)))
print("Unique words: {}".format(len(set(trimmed_words))))

Total words: 23533412
Unique words: 138721


In [6]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [7]:
vocab_to_int, int_to_vocab = create_lookup_tables(trimmed_words)
int_words = [vocab_to_int[word] for word in trimmed_words]

In [8]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
print(max(p_drop.values()))
print(min(p_drop.values()))
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

0.9863050084018772
-5.2627752101870415


In [9]:

def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [10]:

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [11]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [12]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [13]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [14]:

with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
epochs = 50
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                #sim = similarity.eval()
                #for i in range(valid_size):
                #    valid_word = int_to_vocab[valid_examples[i]]
                #    top_k = 8 # number of nearest neighbors
                #    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                #    log = 'Nearest to %s:' % valid_word
                #    for k in range(top_k):
                #        close_word = int_to_vocab[nearest[k]]
                #        log = '%s %s,' % (log, close_word)
                #    print(log)
                print("iteration at 1000")
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/50 Iteration: 100 Avg. Training loss: 5.7098 0.0478 sec/batch
Epoch 1/50 Iteration: 200 Avg. Training loss: 5.7320 0.0434 sec/batch
Epoch 1/50 Iteration: 300 Avg. Training loss: 5.7427 0.0434 sec/batch
Epoch 1/50 Iteration: 400 Avg. Training loss: 5.5874 0.0432 sec/batch
Epoch 1/50 Iteration: 500 Avg. Training loss: 5.5439 0.0433 sec/batch
Epoch 1/50 Iteration: 600 Avg. Training loss: 5.6988 0.0431 sec/batch
Epoch 1/50 Iteration: 700 Avg. Training loss: 5.5804 0.0432 sec/batch
Epoch 1/50 Iteration: 800 Avg. Training loss: 5.4845 0.0433 sec/batch
Epoch 1/50 Iteration: 900 Avg. Training loss: 5.5159 0.0432 sec/batch
Epoch 1/50 Iteration: 1000 Avg. Training loss: 5.5475 0.0433 sec/batch
iteration at 1000
Epoch 1/50 Iteration: 1100 Avg. Training loss: 5.6016 0.0433 sec/batch
Epoch 1/50 Iteration: 1200 Avg. Training loss: 5.5607 0.0434 sec/batch
Epoch 1/50 Iteration: 1300 Avg. Training loss: 5.4790 0.0431 sec/batch
Epoch 1/50 Iteration: 1400 Avg. Training loss: 5.0125 0.0431 sec/bat

Epoch 2/50 Iteration: 11400 Avg. Training loss: 4.1894 0.0433 sec/batch
Epoch 2/50 Iteration: 11500 Avg. Training loss: 3.7545 0.0432 sec/batch
Epoch 2/50 Iteration: 11600 Avg. Training loss: 4.1765 0.0431 sec/batch
Epoch 2/50 Iteration: 11700 Avg. Training loss: 4.2196 0.0434 sec/batch
Epoch 2/50 Iteration: 11800 Avg. Training loss: 4.1328 0.0432 sec/batch
Epoch 2/50 Iteration: 11900 Avg. Training loss: 4.0414 0.0433 sec/batch
Epoch 2/50 Iteration: 12000 Avg. Training loss: 4.0959 0.0431 sec/batch
iteration at 1000
Epoch 2/50 Iteration: 12100 Avg. Training loss: 4.1048 0.0429 sec/batch
Epoch 2/50 Iteration: 12200 Avg. Training loss: 4.1344 0.0433 sec/batch
Epoch 2/50 Iteration: 12300 Avg. Training loss: 3.9143 0.0430 sec/batch
Epoch 2/50 Iteration: 12400 Avg. Training loss: 4.0247 0.0426 sec/batch
Epoch 2/50 Iteration: 12500 Avg. Training loss: 4.1154 0.0431 sec/batch
Epoch 2/50 Iteration: 12600 Avg. Training loss: 4.0989 0.0430 sec/batch
Epoch 2/50 Iteration: 12700 Avg. Training loss

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    sess.run(tf.global_variables_initializer())
    #embed_mat = sess.run(embedding)
    #test = similarity.eval(session=sess)
    #valid_size = 16 # Random set of words to evaluate similarity on.
    #valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    #valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
  
    #valid_examples = np.append(valid_examples, 
    #                           random.sample(range(1000,1000+valid_window), valid_size//2))
    #print(valid_examples)
    word = 'jezusi'
    example = []
    example.append(vocab_to_int[word])
    valid_dataset = tf.constant(example, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    
    normalized_embedding = embedding / norm
    
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    print(valid_embedding)
    similarity2 = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))
    sim2 = similarity2.eval()
    
    
   
    valid_word = word
    top_k = 8
    nearest = (-sim2[0, :]).argsort()[1:top_k+1]
    print(nearest)
    log = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = int_to_vocab[nearest[k]]
        log = '%s %s,' % (log, close_word)
    print(log)
  

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])